In [ ]:
#=
Things to work on:
• Decide how functions will work when L<=ς, or if that's necessary
• Allow initialization for non-square number of particles
• Try out 3 dimensional calculations and plots
• Study nested sampling, the partition function, and thermodynamics
•••• Odd shaped systems
• Temperatures
• Moving system centered on center of mass of particles

Less concern:
• How to waste less space when initializing?
• Ways to speed up graphing
=#

# Run this cell ONCE; it takes time and doesn't need to be run more than once
# Rerun if you shutdown the kernel
using Pkg
Pkg.add("Plots")
using Plots
gr(legend = :none)
using BenchmarkTools

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  99.9 %0.0 %Fetching: [=====>                                   ]  11.6 %>                              ]  24.0 %                         ]  37.1 %.7 %                 ]  55.2 %.9 %    ]  88.3 %.1 % Resolving package versions...
 Installed DataAPI ──────────── v1.4.0
 Installed Conda ────────────── v1.5.0
 Installed Parsers ──────────── v1.0.12
 Installed OrderedCollections ─ v1.3.2
 Installed StaticArrays ─────── v0.12.5
 Installed Contour ──────────── v0.5.6
 Installed GeometryTypes ────── v0.7.10
 Installed StatsBase ────────── v0.32.2
 Installed NaNMath ──────────── v0.3.5
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
  [3da002f7] ↓ ColorTypes v0.10.8 ⇒ v0.9.1
  [5ae59095] ↓ Colors v0.12.4 ⇒ v0.11.2
  [8f4d0f93] ↑ Conda v1.4.1 ⇒ v1.5.0

┌ Info: Recompiling stale cache file /Users/melaniesmacbook/.julia/compiled/v1.1/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184


In [ ]:
#=
Calculating force of particle j on particle i
=#

function comp_force(sqς, xi, xj, yi, yj, Fx, Fy, i, r = (xi - xj)^2 + (yi - yj)^2)
    
    # Check if i and j are too far apart to matter
    if r < sqς
        r = sqrt(r)
        
        # Calculate magnitude of force from particle j
        F = -24*r^(-7) + 48*r^(-13)
        
        # Calculate x and y components of force from j, then add to net force on i
        Fx[i] += (xi - xj) / r * F  # cos(theta) * F
        Fy[i] += (yi - yj) / r * F  # sin(theta) * F 
    end
    
end

In [ ]:
#=
Using Verlet method to get new position and velocity
=#

function newposition(K,L,z,zprev,Fz,dt,vz)
    
    for i in 1:K
    
        znew = 2z[i] - zprev[i] + Fz[i]*dt^2
        vz[i] = (znew - zprev[i]) / (2dt)
        
        # If new position falls out of bounds, pull it in on opposite side
        # Also do this for previous position so it doesn't muck up later calculations
        
        
        if znew >= L
            zprev[i] = z[i] - 2L
            z[i] = znew - 2L
        elseif znew < -L
            zprev[i] = z[i] + 2L
            z[i] = znew + 2L
        else
            zprev[i] = z[i]
            z[i] = znew
        end
        
        
    end
        
end

In [ ]:
#=
Takes each particle's current coordinates and saves them to a small array, 
then saves that small array to a bigger array. It will be used to animate our particles.
Each small array has the information to plot one frame of an animation.
=#

function recordgifpoints(K,x,y,x_for_gif,y_for_gif)
    
    # Initialize for adding array to plot in gif
    temp_x = []
    temp_y = []   
    
    for i in 1:K
        # Create small list of particle positions for one frame
        push!(temp_x, x[i])
        push!(temp_y, y[i]) 
    end

    # Push small lists to large arrays
    push!(x_for_gif, temp_x)
    push!(y_for_gif, temp_y)
    
end

In [ ]:
function image_grid(L,xj,yj)
    #= 
    Creating periodic boundary versions of j's coordinates 
    displaced by +/- 2L in x and/or y directions
    =#
    
    lo_xj = xj - 2*L
    hi_xj = xj + 2*L
    lo_yj = yj - 2*L
    hi_yj = yj + 2*L
    
    pbx = [lo_xj, xj, hi_xj, lo_xj, xj, hi_xj, lo_xj, xj, hi_xj]
    pby = [lo_yj, lo_yj, lo_yj, yj, yj, yj, hi_yj, hi_yj, hi_yj]
    
    return pbx, pby
end

In [ ]:
function ninedots(L,ς,sqς,xi,xj,yi,yj,Fx,Fy,i)
    # Finding nearest version of particle j through periodic boundaries
                
    r = sqς  # Initialize maximum distance of interference
    npx = xj # Initialize nearest periodic version of particle j
    npy = yj # Initialize nearest periodic version of particle j
    
    pbx, pby = image_grid(L,xj,yj)
    
    for k in 1:9

        R = (xi - pbx[k])^2 + (yi - pby[k])^2
        if R < r 
            r = R
            npx = pbx[k]
            npy = pby[k]
            break
        else 
            continue
        end

    end

    comp_force(sqς,xi,npx,yi,npy,Fx,Fy,i,r)
    
end

In [ ]:
function onebox(L,ς,sqς,xi,xj,yi,yj,Fx,Fy,i)
    # Finding nearest version of particle j through periodic boundaries
     
    lbx = xi - L # Lower x boundary for box of side length L centered on particle i
    ubx = xi + L # Upper x boundary for box of side length L centered on particle i
    lby = yi - L # Lower y boundary for box of side length L centered on particle i
    uby = yi + L # Upper y boundary for box of side length L centered on particle i

    npx = xj  # initialize nearest periodic version of x-coordinate of particle j
    npy = yj  # initialize nearest periodic version of y-coordinate of particle j

    
    pbx, pby = image_grid(L,xj,yj)
    
    for k in 1:9
        # Check if the image is outside the box centered on particle i
        if pbx[k]<lbx || pbx[k]>ubx || pby[k]<lby || pby[k]>uby
            continue
        # If the image is inside the box, record the coordinates and don't bother checking others
        else 
            npx = pbx[k]
            npy = pby[k]
            break # Break out of this small loop with pbx and pby to use   
        end

    end

    comp_force(sqς,xi,npx,yi,npy,Fx,Fy,i)
    
end

In [ ]:
function fourbox(L,ς,sqς,xi,xj,yi,yj,Fx,Fy,i)    
    # Finding nearest version of particle j through periodic boundaries
    
    lbx = xi - 2*L # Lower x boundary for box of side length L centered on particle i
    ubx = xi + 2*L # Upper x boundary for box of side length L centered on particle i
    lby = yi - 2*L # Lower y boundary for box of side length L centered on particle i
    uby = yi + 2*L # Upper y boundary for box of side length L centered on particle i
            
    npx = xj  # initialize nearest periodic version of x-coordinate of particle j
    npy = yj  # initialize nearest periodic version of y-coordinate of particle j

    r = sqς  # Maximum value for interference to take place
    
    pbx, pby = image_grid(L,xj,yj)
    
    for k in 1:9

        # Periodic version needs fall within all bounds
        if pbx[k]<lbx || pbx[k]>ubx || pby[k]<lby || pby[k]>uby
            continue
        else 
            # Check if i and periodic version of j are closer than 3
            R = (xi - pbx[k])^2 + (yi - pby[k])^2
            if R < r
                r = R
                npx = pbx[k]
                npy = pby[k]
                break
            end
        end

    end
    
    comp_force(sqς,xi,npx,yi,npy,Fx,Fy,i,r)
    
end

In [ ]:
function fourdots(L,ς,sqς,xi,xj,yi,yj,Fx,Fy,i)    
    # Finding nearest version of particle j through periodic boundaries
    
    npx = xj  # initialize nearest periodic version of x-coordinate of particle j
    npy = yj  # initialize nearest periodic version of y-coordinate of particle j

    r = sqς  # Maximum value for interference to take place
    
    #=
    Begin by creating grid of 4 (or possibly 2) points to compare.
    If either the x coordinates or y coordinates match, only 2 points are tested.
    If i's coord is greater than j's, j's coord will shift in the positive direction.
    Likewise, if i's coord is smaller, j's coord will shift in the negative direction.
    The unshifted particle j will always be tested.
    =#
    
    # sign(i - j) will return 1 if zi > zj, -1 if zi < zj, and 0 if zi == zj
    diff_x = sign(xi - xj)
    diff_y = sign(yi - yj)
    
    shift_xj = xj + diff_x*2*L
    shift_yj = yj + diff_y*2*L
    
    
    if diff_x == 0
        pbx = [xj,xj]
        pby = [yj,shift_yj]
        
        
    elseif diff_y == 0
        pbx = [xj,shift_xj]
        pby = [yj,yj]
        
    else
        pbx = [xj, shift_xj, xj, shift_xj]
        pby = [yj, yj, shift_yj, shift_yj] 
        
    end
    
    
    # Check if i and periodic version of j are closer than 3
    for k in 1:length(pbx)
        R = (xi - pbx[k])^2 + (yi - pby[k])^2
        if R < r
            r = R
            npx = pbx[k]
            npy = pby[k]
            break
        end
    end
    
    comp_force(sqς,xi,npx,yi,npy,Fx,Fy,i,r)
    
end


In [ ]:
function oneof4dots(L,ς,sqς,xi,xj,yi,yj,Fx,Fy,i)    
    # Finding nearest version of particle j through periodic boundaries
    
    lbx = xi - L # Lower x boundary for box of side length L centered on particle i
    ubx = xi + L # Upper x boundary for box of side length L centered on particle i
    lby = yi - L # Lower y boundary for box of side length L centered on particle i
    uby = yi + L # Upper y boundary for box of side length L centered on particle i
    
    npx = xj  # initialize nearest periodic version of x-coordinate of particle j
    npy = yj  # initialize nearest periodic version of y-coordinate of particle j
    
    #=
    Begin by creating grid of 4 (or possibly 2) points to compare.
    If either the x coordinates or y coordinates match, only 2 points are tested.
    If i's coord is greater than j's, j's coord will shift in the positive direction.
    Likewise, if i's coord is smaller, j's coord will shift in the negative direction.
    The unshifted particle j will always be tested.
    =#
    
    # sign(i - j) will return 1 if zi > zj, -1 if zi < zj, and 0 if zi == zj
    diff_x = sign(xi - xj)
    diff_y = sign(yi - yj)
    
    shift_xj = xj + diff_x*2*L
    shift_yj = yj + diff_y*2*L
    
    
    if diff_x == 0
        pbx = [xj,xj]
        pby = [yj,shift_yj]
        
        
    elseif diff_y == 0
        pbx = [xj,shift_xj]
        pby = [yj,yj]
        
    else
        pbx = [xj, shift_xj, xj, shift_xj]
        pby = [yj, yj, shift_yj, shift_yj] 
        
    end
    
    
    
    # Find image of j within box of side length 2L centered on i
    for k in 1:length(pbx)
        
        # Check if the image is outside the box centered on particle i
        if pbx[k]<lbx || pbx[k]>ubx || pby[k]<lby || pby[k]>uby
            continue
            
        # If the image is inside the box, record the coordinates and don't bother checking others
        else 
            npx = pbx[k]
            npy = pby[k]
            break # Break out of this small loop with pbx and pby to use   
        end
        
    end

    comp_force(sqς,xi,npx,yi,npy,Fx,Fy,i)
    
end

In [ ]:
#=
This function creates the nearest periodic version from the getgo by examining which boundaries each
particle is nearest to.
=#

function oneimage(L,ς,sqς,xi,xj,yi,yj,Fx,Fy,i)
    
    #= 
    Find minimum between distance to upper bound, lower bound, and cutoff distance. 
    Use result for creating  nearest periodic version of particle j. Repeat comparison with other coordinates.
    =#
    
    #= 
    In either dimension, there are 3 areas: near lower periodic boundary, near upper one, or near neither of them.
    
    If i coord is much higher than j coord, j will be transformed to be 2L higher.
    If i coord is much lower than j coord, j will be transformed to be 2L lower.
    If i coord and j coord are in about the same area, j coord will not be transformed.
    =#
    
    # Check if xi is within cutoff of boundary
    if (L - abs(xi)) >= ς
        nbxi = 0
    else
        nbxi = sign(xi)
    end
    
    
    # Check if xj is within cutoff of boundary
    if (L - abs(xj)) >= ς
        nbxj = 0
    else
        nbxj = sign(xj)
    end
    

    # Check if yi is within cutoff of boundary
    if (L - abs(yi)) >= ς
        nbyi = 0
    else
        nbyi = sign(yi)
    end
    
    
    # Find nearest boundary to yj
    # Check if yj is within cutoff of boundary
    if (L - abs(yj)) >= ς
        nbyj = 0
    else
        nbyj = sign(yj)
    end
    
    #=
    # This section of code should work even when L < ς
    # Find nearest boundary to xi
    lbxi = L + xi # Distance between left boundary and xi
    ubxi = L - xi # Distance between right boundary and xi
    tmp = min(lbxi, ubxi, ς)
    
    if tmp == lbxi
        nbxi = -1
    elseif tmp == ubxi
        nbxi = 1
    else
        nbxi = 0
    end

    
    # Find nearest boundary to xj
    lbxj = L + xj
    ubxj = L - xj
    tmp = min(lbxj, ubxj, ς)
    
    if tmp == lbxj
        nbxj = -1
    elseif tmp == ubxj
        nbxj = 1
    else
        nbxj = 0
    end
    
    
    # Find nearest boundary to yi
    lbyi = L + yi
    ubyi = L - yi
    tmp = min(lbyi, ubyi, ς)
    
    if tmp == lbyi
        nbyi = -1
    elseif tmp == ubyi
        nbyi = 1
    else
        nbyi = 0
    end
    
    
    # Find nearest boundary to yi
    lbyj = L + yj
    ubyj = L - yj
    tmp = min(lbyj, ubyj, ς)
    
    if tmp == lbyj
        nbyj = -1
    elseif tmp == ubyj
        nbyj = 1
    else
        nbyj = 0
    end
    =#
    
    npx = xj + (abs(nbxi*nbxj))*sign(nbxi-nbxj)*2*L
    npy = yj + (abs(nbyi*nbyj))*sign(nbyi-nbyj)*2*L
    
    
    comp_force(sqς,xi,npx,yi,npy,Fx,Fy,i)
    
end

In [ ]:
#=
Takes the computation function you wish to use as a parameter, computes particle paths
=#

function comp_path(K,L,ς,sqς,x,y,Fx,Fy,chosen_way::Function)
    
    # Calculating net force on each particle one at a time
    
    
    for i in 1:K
        # Initialize forces as zero, we will add on forces from each other particle
        Fx[i] = 0
        Fy[i] = 0
        
        #= 
        If particle i is far from any system boundaries, do force calculations only inside main system.
        =#
        if L-abs(x[i])>=ς && L-abs(y[i])>=ς
                
            for j in 1:K

                # Check if i and j are same particle; particle i can't act on itself
                if i == j
                    continue
        
                else
                    comp_force(sqς,x[i],x[j],y[i],y[j],Fx,Fy,i)
                    
                end
                
            end
         
            
        #= 
        Since particle i is near a boundary, check if particle j 
        is also near any system boundaries. If so do force calculations
        with periodic versions of j. If not, use versions of j within system.
        =#
        else
            
            # Calculate forces from each other particle
            for j in 1:K

                # Check if i and j are same particle; particle i can't act on itself
                if i == j
                    continue
                    
                # If particle j is far from any system boundaries, do force calculations only inside main system.    
                elseif L-abs(x[j])>=ς && L-abs(y[j])>=ς
                    comp_force(sqς,x[i],x[j],y[i],y[j],Fx,Fy,i)
    
                else
                    chosen_way(L,ς,sqς,x[i],x[j],y[i],y[j],Fx,Fy,i) 
                    
                end # if i == j ...
                
            end # for j in 1:K
            
            
        end # if abs(L-x[i])>=3 && abs(L-y[i])>=3 ...
        
    end # for i in 1:K
    
end

In [ ]:
### INITIAL VALUES
# After running each previous cell once, start future runs from this cell or later

dt = 0.02         # time step size
N  = 10           # number of particles on one side
K  = N^2          # total number of particles
ds = 1.3          # grid spacing
L  = 1.5 * N * ds # system size
dr = 0.01         # max deviation from grid position
v0 = 0.1          # max initial velocity magnitude
vxavg = -0.75      # system average velocity in x direction
vyavg = 0.75       # system average velocity in y direction
σ = 1             # Distance unit
ς = 3 * σ         # Max distance where interference is possible
sqς = ς^2         # Square of max distance where interference is possible, useful for our functions
                  # Let sigma and epsilon both be 1.

# Initialize vectors
init_x = []
init_y = []
init_vx = []
init_vy = []
init_xprev = []
init_yprev = []

k = 1 #iterator for creating particle grid

# Create grid of NxN particles
for i in 1:N
    for j in 1:N
        # Create coordinates for particle ij, displace randomly, move so grid center is at origin
        push!(init_x,(i-1)*ds + (rand() - 0.5)*dr - (N-1)/2*ds)
        push!(init_y,(j-1)*ds + (rand() - 0.5)*dr - (N-1)/2*ds)
        
        # Randomize initial velocity for calculating ghost previous position
        push!(init_vx,(rand() - 0.5)v0 + vxavg)
        push!(init_vy,(rand() - 0.5)v0 + vyavg)
        
        # Use initial velocity for initializing ghost previous position
        push!(init_xprev,init_x[k] - init_vx[k] * dt)
        push!(init_yprev,init_y[k] - init_vy[k] * dt)
        
        k += 1
    end
end

init_vals = [init_x, init_y, init_vx, init_vy, init_xprev, init_yprev]

println("This cell has finished running.")

In [ ]:
T = 3000 # number of timesteps to take
s = 10 # Desired number of iterations between data recorded for plots

func_options = [ninedots, onebox, fourbox, 
                fourdots, oneof4dots, oneimage];

plotnames = ["Nine-Atom", "One-Box, Nine Images", "Four-Box, Nine Images", 
            "Four-Atom", "One-Box, Four Images", "One Image"];

checknames = ["nine atom", "one box, nine image", "four-box, nine image", 
            "four atom", "one box, four image", "one image"];

mp4_names = ["ninedots.mp4", "onebox.mp4", "fourbox.mp4", 
             "fourdots.mp4", "onein4dots.mp4", "oneimage.mp4"];

#= 
This is in its own cell so it can be adjusted separately from both initial values and
the various methods of calculating particle paths.
=#

In [ ]:
### Calculate particle paths using each method and record points for plotting

x_for_anim = []
y_for_anim = []

for i in 1:length(func_options)
    
    x = copy(init_vals[1])
    y = copy(init_vals[2])
    vx = copy(init_vals[3])
    vy = copy(init_vals[4])
    xprev = copy(init_vals[5])
    yprev = copy(init_vals[6])

    
    x_anim_tmp = []
    y_anim_tmp = []
    
    # These are for initializing arrays important for static scatterplots
    # x_to_plot = []
    # y_to_plot = []
    # scatterdots = []

    # Initialize force vectors and arrays for plotting
    Fx = zeros(K)
    Fy = zeros(K)

    for n in 1:T

        # Calculating net force on each particle one at a time
        comp_path(K,L,ς,sqς,x,y,Fx,Fy,func_options[i])

        # Updating coordinates and velocities Verlet method
        newposition(K,L,x,xprev,Fx,dt,vx)
        newposition(K,L,y,yprev,Fy,dt,vy)

        # Record the points every few time steps
        if n%s == 0
            recordgifpoints(K,x,y,x_anim_tmp,y_anim_tmp)
            # recordscatterpoints(K,T,n,s,x,y,x_to_plot,y_to_plot,scatterdots)
        end

    end

    push!(x_for_anim, copy(x_anim_tmp))
    push!(y_for_anim, copy(y_anim_tmp))

    println("Check number ", i, " has finished running.")

end

In [16]:
println("This cell is almost done...")

function p(k,i)
    lim = L
    dotsize = 3

    scatter(x_for_anim[k][i], y_for_anim[k][i], markersize = dotsize, 
            xlims = (-lim,lim), ylims = (-lim,lim),
            ylabel = plotnames[k], size = (800,800), aspectratio = 1)
    
end



subplots_anim = @animate for i = 1:length(x_for_anim[1])

    plot(p(1,i), p(2,i), p(3,i), p(4,i), p(5,i), p(6,i))
    
end

# DO NOT RUN THIS UNTIL YOU KNOW WHERE IT WILL SAVE TO
# gif(subplots_anim,"all_methods.gif")

This cell is almost done...


┌ Info: Saved animation to 
│   fn = /mnt/juliabox/all_methods.gif
└ @ Plots /home/jrun/.julia/packages/Plots/UQI78/src/animation.jl:90


Plots.AnimatedGif("/mnt/juliabox/all_methods.gif")

In [17]:
# Calculating paths of particles without recording coordinates for animations

function do_comp_path(init_vals,K,L,dt,ς,sqς,T,chosen_way::Function)
    
    
    # Initialize starting x, y, vx, vy, xprev, and yprev values
    x = copy(init_vals[1])
    y = copy(init_vals[2])
    vx = copy(init_vals[3])
    vy = copy(init_vals[4])
    xprev = copy(init_vals[5])
    yprev = copy(init_vals[6])

    # Initialize force vectors and arrays for plotting
    Fx = zeros(K)
    Fy = zeros(K)
    
    for n in 1:T

        # Calculating net force on each particle one at a time
        comp_path(K,L,ς,sqς,x,y,Fx,Fy,chosen_way)



        # Updating coordinates and velocities Verlet method
        newposition(K,L,x,xprev,Fx,dt,vx)
        newposition(K,L,y,yprev,Fy,dt,vy)

    end
    
    
end

do_comp_path (generic function with 1 method)

In [18]:
# Using @btime to check which method calculates the fastest and uses the least memory.

println("""We'll now check which method calculates paths the fastest.""")

for i in 1:length(func_options)

    k = i  # do_comp_path can't see the value of i from within @btime, but it can see k

    println("\n","The ",checknames[k]," check takes:")
    @btime(do_comp_path(init_vals,K,L,dt,ς,sqς,T,func_options[k]))
    
end


We'll now check which method calculates paths the fastest.

The nine atom check takes:
  5.053 s (130304607 allocations: 3.05 GiB)

The one box, nine image check takes:
  5.018 s (130304607 allocations: 3.05 GiB)

The four-box, nine image check takes:
  5.289 s (130304607 allocations: 3.05 GiB)

The four atom check takes:
  4.464 s (89145707 allocations: 2.06 GiB)

The one box, four image check takes:
  4.550 s (89145707 allocations: 2.06 GiB)

The one image check takes:
  3.430 s (47986807 allocations: 732.23 MiB)


In [19]:
# Creating mp4 files of each animation

println("This cell is almost done...")

function mp4_p(k,i)
    lim = L
    dotsize = 3

    scatter(x_for_anim[k][i], y_for_anim[k][i], markersize = dotsize, 
            xlims = (-lim,lim), ylims = (-lim,lim),
            title = plotnames[k], dpi = 150, aspectratio = 1)
    
end



for k in 1:length(mp4_names)
    
    
    LJ_anim = @animate for i = 1:length(x_for_anim[k])

        plot(mp4_p(k,i))
        
    end
    
    # DO NOT RUN THIS UNTIL YOU KNOW WHERE IT WILL SAVE TO
    # mp4(LJ_anim,mp4_names[k]);
    
end


This cell is almost done...


┌ Info: Saved animation to 
│   fn = /mnt/juliabox/ninedots.mp4
└ @ Plots /home/jrun/.julia/packages/Plots/UQI78/src/animation.jl:90
┌ Info: Saved animation to 
│   fn = /mnt/juliabox/onebox.mp4
└ @ Plots /home/jrun/.julia/packages/Plots/UQI78/src/animation.jl:90
┌ Info: Saved animation to 
│   fn = /mnt/juliabox/fourbox.mp4
└ @ Plots /home/jrun/.julia/packages/Plots/UQI78/src/animation.jl:90
┌ Info: Saved animation to 
│   fn = /mnt/juliabox/fourdots.mp4
└ @ Plots /home/jrun/.julia/packages/Plots/UQI78/src/animation.jl:90
┌ Info: Saved animation to 
│   fn = /mnt/juliabox/onein4dots.mp4
└ @ Plots /home/jrun/.julia/packages/Plots/UQI78/src/animation.jl:90
┌ Info: Saved animation to 
│   fn = /mnt/juliabox/oneimage.mp4
└ @ Plots /home/jrun/.julia/packages/Plots/UQI78/src/animation.jl:90


In [20]:
# Haven't needed to use this since I got the gifs up and running
# Static scatterplot
#=
function recordscatterpoints(K,T,n,s,x,y,x_to_plot,y_to_plot,scatterdots)
    
    for i in 1:K
        # For static scatterplot, add to list of coordinates
        push!(x_to_plot, x[i])
        push!(y_to_plot, y[i])
        
        if T - n < s
            push!(scatterdots,2)  # We want big dots to mark the end of paths
        elseif n == s
            push!(scatterdots,2)  # We want big dots at the beginning of paths, too
        else
            push!(scatterdots,1)  # Small dots everywhere else
        end
    end
    
end
=#
#scatter(x_to_plot, y_to_plot, markersize = scatterdots)

In [21]:
# Returns animated gif using arrays of coordinates and system size
#=
function gifplot(x_for_gif,y_for_gif,L)
    
    # Animated gif
    @gif for i = 1:length(x_for_gif)
        scatter(x_for_gif[i], y_for_gif[i], markersize = 2, 
            xlims = (-L,L),
            ylims = (-L,L),
            dpi = 120,
            aspectratio = 1)
    end
    
end
=#

In [22]:
#=
Using Verlet method to get new position and velocity
=#

function newpositionodd(K,L,x,xprev,Fx,dt,vx,y,yprev,Fy,vy,RL,UL)
    
    for i in 1:K
    
        xnew = 2x[i] - xprev[i] + Fx[i]*dt^2
        ynew = 2y[i] - yprev[i] + Fy[i]*dt^2
        # vx[i] = (xnew - xprev[i]) / (2dt)
        
        x_trans = 0
        y_trans = 0
        
        # If new position falls out of bounds, pull it in on opposite side
        # Also do this for previous position so it doesn't muck up later calculations
        
        
        # If y is transported across a boundary, x will need to transform too
        if ynew + UL*xnew >= L
            y_trans += -2L
            x_trans += RL*2L
            
        elseif ynew + UL*xnew <= -L
            y_trans += 2L
            x_trans += -RL*2L

        end
        
        
        
        if xnew + RL*ynew >= L
            x_trans += -2L
            y_trans += UL*2L
            
        elseif xnew + RL*ynew <= -L 
            x_trans += 2L
            y_trans += -UL*2L
            
        end
        
        
        
        xprev[i] = x[i] + x_trans
        x[i] = xnew + x_trans
        
        yprev[i] = y[i] + y_trans
        y[i] = ynew + y_trans

        
        #=
        if xnew >= L
            xprev[i] = x[i] - 2L
            x[i] = xnew - 2L
        elseif xnew < -L
            xprev[i] = x[i] + 2L
            x[i] = xnew + 2L
        else
            xprev[i] = x[i]
            x[i] = xnew
        end
        =#
        
    end
        
end

newpositionodd (generic function with 1 method)

In [23]:
#=
This function creates the nearest periodic version from the getgo by examining which boundaries each
particle is nearest to.
=#

function oddshape1(L,ς,sqς,xi,xj,yi,yj,Fx,Fy,i,RL,UL)
    #= 
    A parallelogram with upper border y = L, lower border y = -L, 
    right border y = 2(x - L), and left border y = 2(x + L)
    =#
    
    #= 
    Find minimum between distance to upper bound, lower bound, and cutoff distance. 
    Use result for creating  nearest periodic version of particle j. Repeat comparison with other coordinates.
    =#
    
    #= 
    In either dimension, there are 3 areas: near lower periodic boundary, near upper one, or near neither of them.
    
    If i coord is much higher than j coord, j will be transformed to be 2L higher.
    If i coord is much lower than j coord, j will be transformed to be 2L lower.
    If i coord and j coord are in about the same area, j coord will not be transformed.
    =#
    
    # Check if xi is within cutoff of boundary
    if L - abs(xi + RL*yi) >= ς
        nbxi = 0
    else
        nbxi = sign(xi)
    end
    
    
    # Check if xj is within cutoff of boundary
    if  L - abs(xj + RL*yj) >= ς
        nbxj = 0
    else
        nbxj = sign(xj)
    end
    

    # Check if yi is within cutoff of boundary
    if (L - abs(yi + UL*xi)) >= ς
        nbyi = 0
    else
        nbyi = sign(yi)
    end
    
    
    # Find nearest boundary to yj
    # Check if yj is within cutoff of boundary
    if (L - abs(yj + UL*xj)) >= ς
        nbyj = 0
    else
        nbyj = sign(yj)
    end
    
    
    npx = xj + abs(nbxi*nbxj)*sign(nbxi-nbxj)*2*L - RL*abs(nbyi*nbyj)*sign(nbyi-nbyj)*2*L
    npy = yj + abs(nbyi*nbyj)*sign(nbyi-nbyj)*2*L - UL*abs(nbxi*nbxj)*sign(nbxi-nbxj)*2*L
    
    
    comp_force(sqς,xi,npx,yi,npy,Fx,Fy,i)
    
end

oddshape1 (generic function with 1 method)

In [24]:
#=
Takes the computation function you wish to use as a parameter, computes particle paths
=#

function comp_oddpath1(K,L,ς,sqς,x,y,Fx,Fy,RL,UL)
    
    # Calculating net force on each particle one at a time
    
    
    for i in 1:K
        # Initialize forces as zero, we will add on forces from each other particle
        Fx[i] = 0
        Fy[i] = 0
        
        #= 
        If particle i is far from any system boundaries, do force calculations only inside main system.
        =#
        if (L - abs(x[i] + RL*y[i]))>=ς && (L - abs(y[i] + UL*x[i]))>=ς
                
            for j in 1:K

                # Check if i and j are same particle; particle i can't act on itself
                if i == j
                    continue
        
                else
                    comp_force(sqς,x[i],x[j],y[i],y[j],Fx,Fy,i)
                    
                end
                
            end
         
            
        #= 
        Since particle i is near a boundary, check if particle j 
        is also near any system boundaries. If so do force calculations
        with periodic versions of j. If not, use versions of j within system.
        =#
        else
            
            # Calculate forces from each other particle
            for j in 1:K

                # Check if i and j are same particle; particle i can't act on itself
                if i == j
                    continue
                    
                # If particle j is far from all system boundaries, do force calculations only inside main system.    
                elseif (L - abs(x[j] + RL*y[j]))>=ς && (L - abs(y[j] + UL*x[j]))>=ς
                    comp_force(sqς,x[i],x[j],y[i],y[j],Fx,Fy,i)
    
                else
                    oddshape1(L,ς,sqς,x[i],x[j],y[i],y[j],Fx,Fy,i,RL,UL) 
                    
                end # if i == j ...
                
            end # for j in 1:K
            
            
        end # if abs(L-x[i])>=3 && abs(L-y[i])>=3 ...
        
    end # for i in 1:K
    
end

comp_oddpath1 (generic function with 1 method)

In [25]:
# Calculating particle paths inside parallelogram system of area (2L)^2


"""Do not exceed ONE for either of these values, or else oneimage function won't work"""
RL = 0.0 # negative means right-leaning, positive means left leaning
UL = -0.95 # negative means low-high slant, positive means high-low slant
"""Either RL or UL must be ZERO"""

#=
Paralellogram boundaries have the following equations:
Upper: y + UL*x =  L
Lower: y + UL*x = -L
Right: x + RL*y =  L
 Left: x + RL*y = -L
=#

x = copy(init_vals[1])
y = copy(init_vals[2])
vx = copy(init_vals[3])
vy = copy(init_vals[4])
xprev = copy(init_vals[5])
yprev = copy(init_vals[6])


x_for_gif = []
y_for_gif = []

# These are for initializing arrays important for static scatterplots
# x_to_plot = []
# y_to_plot = []
# scatterdots = []

# Initialize force vectors and arrays for plotting
Fx = zeros(K)
Fy = zeros(K)

for n in 1:T

    # Calculating net force on each particle one at a time
    comp_oddpath1(K,L,ς,sqς,x,y,Fx,Fy,RL,UL)

    # Updating coordinates and velocities Verlet method
    newpositionodd(K,L,x,xprev,Fx,dt,vx,y,yprev,Fy,vy,RL,UL)

    # Record the points every few time steps
    if n%s == 0
        recordgifpoints(K,x,y,x_for_gif,y_for_gif)
        # recordscatterpoints(K,T,n,s,x,y,x_to_plot,y_to_plot,scatterdots)
    end

end


println("This cell has finished running.")

This cell has finished running.


In [26]:
xcorner1 = -L + RL*L
xcorner2 = -L - RL*L
xcorner3 = -xcorner1
xcorner4 = -xcorner2

ycorner1 = -L + UL*L
ycorner2 =  L + UL*L
ycorner3 = -ycorner1
ycorner4 = -ycorner2

xlim = L + abs(RL)*L
ylim = L + abs(UL)*L

LJ_paralellogram = @animate for i = 1:length(x_for_gif)
        scatter(x_for_gif[i], y_for_gif[i], markersize = 3, 
            xlims = (-xlim,xlim),
            ylims = (-ylim,ylim),
            dpi = 150,
            aspectratio = 1)
        plot!([xcorner1,xcorner2,xcorner3,xcorner4,xcorner1],[ycorner1,ycorner2,ycorner3,ycorner4,ycorner1])
    end
# DO NOT RUN THIS UNTIL YOU KNOW WHERE IT WILL SAVE TO
# mp4(LJ_paralellogram,"parallelogram.mp4");

┌ Info: Saved animation to 
│   fn = /mnt/juliabox/parallelogram.mp4
└ @ Plots /home/jrun/.julia/packages/Plots/UQI78/src/animation.jl:90
